In [2]:
!pip install schedule

In [8]:
import psycopg2
import json
from datetime import datetime

#connect to Supabase PostgreSQL database
def connect_db():
    return psycopg2.connect(
        host="aws-0-ap-south-1.pooler.supabase.com",
        database="postgres",
        user="postgres.erkmglmhqgsdyrjniozo",
        password="xOdgFwQ4O8dbqPVZ",
        port="6543"
    )

#fetch metrics and insights
def fetch_metrics_and_store():
    connection = connect_db()
    cursor = connection.cursor()

    #calculate average CTR
    cursor.execute("""
    SELECT
        search_date,
        AVG(click_through_rate) AS average_ctr
    FROM
        search_clicks
    GROUP BY
        search_date;
    """)
    avg_ctr = cursor.fetchall()

    #fetch top 5 queries
    cursor.execute("""
    SELECT
        search_query,
        click_through_rate
    FROM
        search_clicks
    ORDER BY
        click_through_rate DESC
    LIMIT 5;
    """)
    top_queries = cursor.fetchall()

    #fetch low-performing queries
    cursor.execute("""
    SELECT
        search_query,
        impressions,
        clicks
    FROM
        search_clicks
    WHERE
        impressions > 500 AND clicks < 50;
    """)
    low_perf_queries = cursor.fetchall()

    #insert into search_insights table
    cursor.execute("""
    INSERT INTO search_insights (insight_date, average_ctr, top_queries, low_performance_queries)
    VALUES (%s, %s, %s, %s)
    """, (
        datetime.now().date(),
        avg_ctr[0][1] if avg_ctr else None,
        json.dumps([{"query": row[0], "ctr": row[1]} for row in top_queries]),
        json.dumps([{"query": row[0], "impressions": row[1], "clicks": row[2]} for row in low_perf_queries])
    ))
    connection.commit()

    cursor.close()
    connection.close()

#run the task once
if __name__ == "__main__":
    fetch_metrics_and_store()
    print("Metrics and insights fetched and stored successfully.")


Metrics and insights fetched and stored successfully.
